In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# from einops import rearrange, repeat
# import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
# from train import getSamples, getUpSamples, getSpacing


In [4]:
name = r'building'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (950/971)

In [ ]:
sigmoid_ = nn.Sigmoid()

